VERSION 1.0 ON 28072022 - 22.52

VERSION 1.0.01 ON 29072022 - 00.14 - adjusted output .pdf image size

I HAVE BEEN ALLOTTED 80 GRB FILES (NAMES STORED IN THE VARIABLE $folders$)

OUT OF THOSE 80 FILES, 22 ARE MASTERSAMPLE FILES SO THEY HAVE BEEN IGNORED AS PER INSTRUCTIONS

THE REMAINING 52 FILES ARE WITHOUT DUPLICATE BANDS IN THE SAME TIME AND/OR LIMITING MAGNITUDES

ANY POINTS SEEN OVERLAPPING ARE SIMPLY VERY CLOSE TO EACH OTHER

In [1]:
import plotly.express as px
import pandas as pd
import numpy as np
import kaleido
from PyPDF2 import PdfFileMerger

In [2]:
#FILE PATHS
systempath = "/media/darkwake/VIB2/"
inputfilepath = systempath + "GRB/ALL_LCS_FOR_500/"
output_file_path = systempath + "task2/v_converted_filtered_files/"
output_pdf_path = systempath + "GRB/task2/v_out_pdfs/"

#THE 80 FILES ALOTTED TO ME
folders = "140423A ,140428A ,140430A ,140506A ,140508A ,140509A ,140512A ,140515A ,140518A ,140606B ,140614A ,140620A ,140623A ,140629A ,140703A ,140710A ,140801A ,140808A ,140903A ,140907A ,141004A ,141028A ,141109A ,141121A ,141220A ,141221A ,141225A ,150101B ,150120B ,150206A ,150301B ,150314A ,150323A ,150403A ,150413A ,150423A ,150514A ,150518A ,150727A ,150818A ,150821A ,150910A ,150915A ,151021A ,151027A ,151027B ,151029A ,151031A ,151111A ,151112A ,151215A ,160117B ,160121A ,160131A ,160203A ,160227A ,160228A ,160314A ,160410A ,160425A ,160509A ,160623A ,160625B ,160629A ,160804A ,160821B ,161014A ,161017A ,161023A ,161108A ,161117A ,161129A ,161219B ,170113A ,170202A ,170214A ,170405A ,170428A ,170519A"

#22 MASTERSAMPLE FILES WITHIN MY ALOTTED FILES
masterfolders = "140423A,140430A,140801A,140907A,141121A,141221A,150323A,150413A,150910A,151027A,151029A,160131A,160227A,160314A,160425A,160804A,161014A,161023A,161219B,170113A,170405A"
folders = folders.split(" ,")
masterfolders = masterfolders.split(",")

#REMOVING THE MASTERSAMPLE FILES FROM THE LIST OF FILES TO BE SCANNED
for i in masterfolders:
    folders.remove(i)
filenames = [i + "_converted_flux" for i in folders]

FUNCTION TO REMOVE POINTS WITH 0 ERROR(LIMITING VALUE ERROR) AND CORRUPTED DATA

In [3]:
def zero_error(light):
    for i in range(len(light)):
        if light.flux_err.values[i] == 0.0 or np.log10(light.flux_err.values[i]) > 50  :
            light.flux_err.values[i] = None
        if type(light.index.values[i]) == str:
            light.flux_err.values[i] = None
    light.dropna(inplace=True)       

IF AN OBSERVATION HAS BEEN DONE IN MULTIPLE BANDS AT THE SAME TIME,

THIS METHOD REMOVES ALL THE BANDS EXCEPT R / r / r' BANDS FROM THAT OBSERVATION

In [4]:
def vertical(light):
    disc_time = list(set(light.index.values))
    disc_time.sort()
    val_counts = list(light.index.value_counts(ascending=True))
    for i in range(len(disc_time)):
        if val_counts[i] > 1:
            for j in range(len(light)):
                if light.index.values[j] == disc_time[i] and not(light.band.values[j] in ("R","r", "r'", "R'")):
                    light.band.values[j] = None

    light.dropna(inplace=True)

THE FOLLOWING METHOD CLEANS A GIVEN INPUT FILE AND STORES THE CLEANED DATA FILE IN $task2/v$ _ $converted$ _ $filtered$ _ $files$ FOLDER

In [5]:
def filter(f2):
     filename = inputfilepath + file
     filename2 = f2.replace( "_flux","_filtered_flux")
     try:
          light = pd.read_csv(filename+".txt", sep=r"\t+|\s+", engine="python", names= ["time_sec","flux","flux_err","band","source"],
         header=0, index_col=0)
          zero_error(light)
          vertical(light)
        
          light.to_csv("task2/v_converted_filtered_files/"+filename2 + ".txt")
          return light

     except:
          light = pd.read_csv(filename+".txt", sep=r"\t+|\s+", engine="python", names= ["time_sec","flux","flux_err","band","source"],
         header=0, index_col=0)
          zero_error(light)
          vertical(light)
        
          light.to_csv("task2/v_converted_filtered_files/"+filename2 + ".txt")
          return light


FUNCTION TO PLOT A GIVEN DATABASE AND STORE THE OUTPUT PLOT IN $task2/v$ _ $out$ _ $pdfs$ FOLDER

In [6]:
def plot(light, f2):
    #filename2 = file + "_not_done"
    filename2 = f2.replace( "_flux","_filtered_flux")
    filename = f2
    if len(light) != 0:
        try:
            fig = px.scatter(
                        data_frame=light,
                        x=np.log10(light.index.values),
                        y=np.log10(light[1].values),
                        error_y= light[2].values / (light[1].values * np.log(10)),
                        color=light[3].values,
                        hover_data=[4],
                        title=filename2,width=800, height=400
                    )

            fig.update_layout(
                        xaxis_title='log Time (s)',
                        yaxis_title='log Flux (erg cm-2 s-1)'
                    )
            fig.write_image(output_pdf_path + filename2+".pdf")

            
        except:
            fig = px.scatter(
                data_frame=light,
                x=np.log10(light.index),
                y=np.log10(light['flux'].values),
                error_y= light['flux_err'].values / (light['flux'].values * np.log(10)),
                color=light['band'].values,
                hover_data=['source'],
                title=filename2, width=1600, height=800
            )
            fig.update_layout(
                xaxis_title='log Time (s)',
                yaxis_title='log Flux (erg cm-2 s-1)'
            )
        #fig.show()
        fig.write_image(output_pdf_path + filename2+".pdf")
    else:
        fig = px.scatter()
        fig.write_image(output_pdf_path + filename2+".pdf")

In [7]:
#FUNCTION TO CONCATANATE N DATA FRAMES INTO A SINGLE DATAFRAME AND RETURN IT
def concat(*args):
    return pd.concat([x for x in args if not x.empty])

THE FOLLOWING CELL ITERATES THROUGH THE 58(AFTER REMOVING 22 MASTERSAMPLE FILES FROM THE 80) FILES ALLOTTED TO ME

IT ALSO STORES THE ENTIRE DATA IN THE DATABASE $all$ _ $light$

In [8]:
i=0
all_light = pd.DataFrame()
for file in filenames:
    i+=1
    plot(filter(file), file)
    all_light = concat(all_light, filter(file))


In [9]:
#PLOTTING THE ENTIRE DATA AND STORING THE CONCATANATED DATABASE AS $entire$ _ $data.txt$
plot(all_light, "entire_data")
all_light.to_csv("task2/v_converted_filtered_files/entire_data.txt")

THE FOLLOWING CELL MERGES ALL THE OUTPUT PDF FILES INTO A SINGLE PDF FILE: $ALL$ _ $LCS$ _ $FOR$ _ $500$ _ $filtered$ _ $merged.pdf$

In [10]:
pdfs = [output_pdf_path + file.replace( "_flux","_filtered_flux.pdf")  for file in filenames]
merger = PdfFileMerger()

for pdf in pdfs:
    merger.append(pdf)
    
merger.write(output_pdf_path + "ALL_LCS_FOR_500_filtered_merged.pdf")
#merger.write(output_pdf_path + "merged_not_done.pdf")
merger.close()